## IMPORT TWEEPY

In [ ]:
import tweepy
import csv
import re
from wordcloud import WordCloud

#Twitter API credentials
consumer_key = 'SY1TACPOFQ1CBVmO4OQhw9lpX'
consumer_secret = 'fx4fFoA71CGdtwYUmjr5mnXUAIkHbmOvmw91h5WSi20zMirnYp'
access_token = '1108216575935430656-mbqcloTcgc3WFDMvEvwL1WfT34GhQR'
access_token_secret = 'sz4rlbfZiLZo1612LzAss7VwdGUIP7zsFS7B5EcuAbzNt'

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
#         print ("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

#         print ("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[#tweet.id_str, tweet.created_at, 
        tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv
    with open('%s_tweets.csv'% screen_name, 'w') as f:
        writer = csv.writer(f)
#         writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    tags = []
    with open('%s_tweets.csv'% screen_name) as csvfile:
        tweetsCSV = csv.reader(csvfile, delimiter = ',')
        for row in tweetsCSV:
            if(row):
                text = row[0]
                tag = re.findall(r'\B#(\w*[a-zA-Z]+\w*)', text)
                if(tag):
                    tags.append(tag)
    text = ""
    for i in range(0, len(tags)):
        text = text + '\n' + ' '.join(tags[i])

    # print(text)

    if(text):
        wordcloud = WordCloud().generate(text)

        %pylab inline
        import matplotlib.pyplot as plt
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")

#     pass


## Get hashtags

In [ ]:
#use the account name to get the word cloud
get_all_tweets("@MaribyrnongCC")   